In [1]:
!pip install llama-index>=0.9.31 pinecone-client>=3.0.0
!pip install pymupdf
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-embeddings-cohere

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


: 

In [2]:
import pandas as pd
import os
import fitz

from llama_index.core import Document, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.node_parser import SentenceSplitter

from pinecone import Pinecone, ServerlessSpec

links to get your api keys:

https://www.pinecone.io/

https://dashboard.cohere.com/

In [3]:
# https://dashboard.cohere.com/
cohere_api_key = "...."

# svuj vlastni ziskate na https://app.pinecone.io/
pinecone_api_key = ".."

os.environ["COHERE_API_KEY"] = cohere_api_key
os.environ["PINECONE_API_KEY"] = pinecone_api_key


In [4]:
# with input_typ='search_query'
model = CohereEmbedding(
    api_key=cohere_api_key,
    model_name="embed-english-v3.0"
)

embeddings = model.get_text_embedding("Hello CohereAI!")
len(embeddings)

1024

In [5]:
pdf_names = [
             "Studijni-a-zkusebni-rad-Vysoke-skoly-ekonomicke-v-Praze_5.pdf"
             ]

In [6]:
fitz

<module 'fitz' from '/home/tomas/pythonProjects/Projects/basicRag/RagDemo/venv/lib/python3.10/site-packages/fitz/__init__.py'>

In [8]:
pdf_texts = []

for name in pdf_names:
    with fitz.open(name) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
        pdf_texts.append(text)

pdf_texts[0][:100]

' \n \nStudijní a zkušební řád Vysoké školy \nekonomické v Praze \nplatný předpis \nMinisterstvo školství,'

In [9]:
# Create nodes from a document
documents = [Document(text=t) for t in pdf_texts]

parser = SentenceSplitter(chunk_size=256)
nodes = parser.get_nodes_from_documents(documents)
print(len(nodes))
print(nodes[0])

363
Node ID: 274fdde2-f1ce-45dc-b9b0-e1b20569df33
Text: Studijní a zkušební řád Vysoké školy  ekonomické v Praze  platný
předpis  Ministerstvo školství, mládeže a tělovýchovy registrovalo
podle § 36 odst. 2 zákona č. 111/1998  Sb., o vysokých školách a o
změně a doplnění dalších zákonů (zákon o vysokých školách), dne  18.
července 2017 pod čj. MSMT-19508/2017 Studijní a zkušební řád Vysoké
školy ekon...


# Create a new pinecone index

In [10]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.vector_stores.pinecone import PineconeVectorStore

/home/tomas/pythonProjects/Projects/basicRag/RagDemo/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
Settings.embed_model = model
Settings

_Settings(_llm=None, _embed_model=CohereEmbedding(model_name='embed-english-v3.0', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f23509fb4f0>, num_workers=None, embeddings_cache=None, api_key='TsccYDkDvnchJTGB6ppuIpGLbkWE3cJZcXtbfa5f', base_url=None, truncate='END', input_type=None, embedding_type='float'), _callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f23509fb4f0>, _tokenizer=None, _node_parser=None, _prompt_helper=None, _transformations=None)

In [12]:
pc = Pinecone(api_key=pinecone_api_key)
new_index_name = "rag-demo"
pc.create_index(
    name=new_index_name,
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

{
    "name": "rag-demo",
    "metric": "cosine",
    "host": "rag-demo-94ury7a.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [13]:
pinecone_index = pc.Index(new_index_name)
pinecone_index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [14]:
pinecone_vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=pinecone_vector_store)
index = VectorStoreIndex(
    nodes, storage_context=storage_context
)

Upserted vectors: 100%|██████████| 363/363 [00:03<00:00, 109.89it/s]


# Let's query our vector database!

In [15]:
vector_store = PineconeVectorStore(pinecone_index=pc.Index(new_index_name))
retriever = VectorStoreIndex.from_vector_store(vector_store).as_retriever(similarity_top_k=5)

In [ ]:
response = retriever.retrieve("Kdy mohu opakovat státní zkoušku?")

for i, result in enumerate(response):
    print(f"{i+1}. Text: {result.node.text}")
    print(f"   Similarity: {result.score:.4f}\n")


### TO UDELA RETRIEVAL


1. Text: Opakování se může 
konat nejdříve za čtyři měsíce ode dne neúspěšné obhajoby a to podle podmínek čl. 27. 
12. O průběhu obhajoby disertační práce je veden protokol, v němž je uveden název 
doktorského studijního programu, jména zúčastněných členů komise pro obhajobu 
disertační práce a oponentů, dále všechny závažné skutečnosti při obhajobě. V závěru 
protokolu je zapsán výsledek hlasování. Protokol podepisuje předseda komise, 
v případě jeho neúčasti zástupce předsedy.
   Similarity: 0.4469

2. Text: Opakování dílčích částí státní závěrečné zkoušky s výjimkou obhajoby 
závěrečné práce je možné nejdříve 1 měsíc ode dne neúspěšného konání zkoušky. 
Opakování obhajoby (pokud je dílčí částí státní závěrečné zkoušky), je možné nejdříve 
3 měsíce ode dne neúspěšného konání zkoušky. Zkušební komise může doporučit 
způsob, rozsah a formu přepracování závěrečné práce. Student může k opakované 
obhajobě předložit stejnou nebo upravenou práci, nebo při dodržení postupu podle čl.
   Simi

In [ ]:
# do posledni bunky:
# GPT PROMPT:


# Based on this code, use a free model from Hugging face to generate a response based on the retrieval step
# response = retriever.retrieve("Jak ziskam stipendium")

# for i, result in enumerate(response):
#     print(f"{i+1}. Text: {result.node.text}")
#     print(f"   Similarity: {result.score:.4f}\n")

